In [1]:
import pandas as pd 
import numpy as np
import random
result = pd.read_csv('../data/3_before2016.csv').dropna()
X_train = result.loc[:,'actq':'lastactual1']
y_train = result.loc[:,'actual']
anlest_train = result.loc[:,'meanest']
N_train = result.shape[0]
result1 = pd.read_csv('../data/3_after2016.csv').dropna()
X_test = result1.loc[:,'actq':'lastactual1']
y_test = result1.loc[:,'actual']
anlest_test = result1.loc[:,'meanest']
N_test = result1.shape[0]

In [2]:
result.shape, result1.shape

((21580, 28), (5845, 28))

In [3]:
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

scaler = StandardScaler()
scaler.fit(X_train)

reg = linear_model.LinearRegression()
reg.fit(scaler.transform(X_train), y_train)

y_lr = reg.predict(scaler.transform(X_test))
ymean = [np.mean(y_test) for i in range(len(y_test))]
mse_analyst = mean_squared_error(anlest_test, y_test)
mse_lr = mean_squared_error(y_lr, y_test)
r2_analyst = r2_score(y_test, anlest_test)
r2_lr = r2_score(y_test, y_lr)
r2 = r2_score(y_test, ymean)
print(mse_analyst, mse_lr, r2_analyst, r2_lr, r2)

0.1090111661283148 1.072487171961861 0.9634181608361052 0.6400960137985481 0.0


In [4]:
def withinten(y_true, y_pred):
    sum1 = 0
    for i in range(len(y_pred)):
        if(y_pred[i] < 1.1*y_true[i] and y_pred[i] > 0.9*y_true[i]):
            sum1+=1
    return sum1/len(y_pred)
withinten(list(y_test), list(anlest_test)), withinten(list(y_test), y_lr), withinten(list(y_test), ymean)

(0.6278870829769033, 0.2937553464499572, 0.10248075278015398)

In [5]:
from sklearn.preprocessing import PolynomialFeatures
X_new = pd.read_csv('../data/3_before2016.csv').dropna().sample(frac=1) #.sort_values(by='fyearq' ,ascending=True)
X_train = X_new.loc[:, 'actq':'lastactual1']
y_train = X_new.loc[:, 'actual']

result1 = pd.read_csv('../data/3_after2016.csv').dropna()
X_test = result1.loc[:,'actq':'lastactual1']
y_test = result1.loc[:,'actual']

poly = PolynomialFeatures(2)
X_train_op = poly.fit_transform(X_train)

In [6]:
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler

X_ridge = X_train_op
N = X_ridge.shape[0]
train = int(0.8*N)
valid = N - train
X_ridge_train = X_ridge[:train]
y_ridge_train = y_train[:train]
X_ridge_valid = X_ridge[train:]
y_ridge_valid = y_train[train:]

scaler1 = StandardScaler()
scaler1.fit(X_ridge_train)
X_ridge_train = scaler1.transform(X_ridge_train)
X_ridge_train[:,0] = np.ones(X_ridge_train.shape[0])
X_ridge_valid = scaler1.transform(X_ridge_valid)
X_ridge_valid[:,0] = np.ones(X_ridge_valid.shape[0])

for lbd in [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]:
    clf = Ridge(alpha=lbd, solver='lsqr')
    clf.fit(X_ridge_train, y_ridge_train)
    print("lambda=", lbd, ";score=", clf.score(X_ridge_valid, y_ridge_valid))

lambda= 0.0001 ;score= 0.7589284098058362
lambda= 0.001 ;score= 0.7589286902518838
lambda= 0.01 ;score= 0.7589314943479163
lambda= 0.1 ;score= 0.7589594989044818
lambda= 1 ;score= 0.759235944164792
lambda= 10 ;score= 0.7616766801213808
lambda= 100 ;score= 0.7705638737626799
lambda= 1000 ;score= 0.7672268817768481
lambda= 10000 ;score= 0.6862934107411782
lambda= 100000 ;score= 0.43246868097996916


In [16]:
from sklearn.pipeline import make_pipeline

clf = make_pipeline(PolynomialFeatures(2), StandardScaler(), Ridge(alpha=10000))
clf.fit(X_train, y_train)
y_pred_clf = clf.predict(X_test)
clf_loss = mean_squared_error(y_test, y_pred_clf)
r2_clf = r2_score(y_test, y_pred_clf)
print(clf_loss, r2_clf, withinten(list(y_test), y_pred_clf))

1.8084367834571533 0.3931269070855544 0.2135158254918734
